# Proyecto 03 - Procesamiento del Lenguaje Natural

## Dataset: The Multilingual Amazon Reviews Corpus

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Es un archivo .zip que contiene tres documentos. Más información sobre el dataset [aquí](https://registry.opendata.aws/amazon-reviews-ml/). Es importante que tengas en cuenta la [licencia](https://docs.opendata.aws/amazon-reviews-ml/license.txt) de este dataset.**

### Exploración de datos y Procesamiento del Lenguaje Natural

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Considera que hasta que no hayas aplicado las herramientas de Procesamiento del Lenguaje Natural vistas, será difícil completar este análisis. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿qué palabras están asociadas a calificaciones positivas y qué palabras a calificaciones negativas?

### Machine Learning

Implementa un modelo que, dada la crítica de un producto, asigne la cantidad de estrellas correspondiente. **Para pensar**: ¿es un problema de Clasificación o de Regresión?

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.
1. Intenta responder la pregunta: ¿Qué información está usando el modelo para predecir?

**Recomendación:** si no te resulta conveniente trabajar en español con NLTK, te recomendamos que explores la librería [spaCy](https://spacy.io/).

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Valdrá la pena convertir el problema de Machine Learning en un problema binario? Es decir, asignar únicamente las etiquetas Positiva y Negativa a cada crítica y hacer un modelo que, en lugar de predecir las estrellas, prediga esa etiqueta. Pensar en qué situación puede ser útil. ¿Esperas que el desempeño sea mejor o peor?
1. ¿Hay algo que te gustaría investigar o probar?

### **¡Tómate tiempo para investigar y leer mucho!**

In [1]:
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [2]:
dataset = pd.read_json('dataset_es_train.json', lines = True)
dataset.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore
3,es_0359921,product_es_0656090,reviewer_es_0224702,1,"No entro en descalificar al vendedor, solo pue...",PRODUCTO NO RECIBIDO.,es,wireless
4,es_0068940,product_es_0662544,reviewer_es_0224827,1,Llega tarde y co la talla equivocada,Devuelto,es,shoes


In [3]:
dataset.shape

(200000, 8)

In [4]:
dataset.drop_duplicates(inplace = True) #no hay datos duplicados ni faltantes
dataset.shape

(200000, 8)

In [5]:
dataset.isna().sum()

review_id           0
product_id          0
reviewer_id         0
stars               0
review_body         0
review_title        0
language            0
product_category    0
dtype: int64

In [6]:
dataset.language.value_counts() #solo reseñas en español

es    200000
Name: language, dtype: int64

In [7]:
dataset.product_category.value_counts() 

home                        26962
wireless                    25886
toy                         13647
sports                      13189
pc                          11191
home_improvement            10879
electronics                 10385
beauty                       7337
automotive                   7143
kitchen                      6695
apparel                      5737
drugstore                    5513
book                         5264
furniture                    5229
baby_product                 4881
office_product               4771
lawn_and_garden              4237
other                        3937
pet_products                 3713
personal_care_appliances     3573
luggage                      3328
camera                       3029
shoes                        2754
digital_ebook_purchase       1843
video_games                  1733
jewelry                      1598
musical_instruments          1530
watch                        1490
industrial_supplies          1482
grocery       

In [8]:
dataset.stars.value_counts()

1    40000
2    40000
3    40000
4    40000
5    40000
Name: stars, dtype: int64

In [9]:
dataset.product_id.value_counts()

product_es_0261843    8
product_es_0741712    7
product_es_0808483    7
product_es_0801630    7
product_es_0874126    7
                     ..
product_es_0412939    1
product_es_0307324    1
product_es_0756188    1
product_es_0233185    1
product_es_0301918    1
Name: product_id, Length: 150938, dtype: int64

In [10]:
dataset.reviewer_id.value_counts() #product id y reviewer id no deberian coincidir?

reviewer_es_0437514    7
reviewer_es_0588051    7
reviewer_es_0431022    7
reviewer_es_0659252    6
reviewer_es_0961357    6
                      ..
reviewer_es_0298010    1
reviewer_es_0953900    1
reviewer_es_0749245    1
reviewer_es_0898211    1
reviewer_es_0435550    1
Name: reviewer_id, Length: 179076, dtype: int64

Elijo una instancia al azar y veo el review boby

In [11]:
index_random = np.random.randint(0,high = dataset.shape[0])
descripcion = dataset.iloc[index_random].review_body
print(index_random, descripcion)

83031 Se me ha fundido una el mismo día que las he puesto. Iluminar, iluminan bastante bien pero estoy algo decepcionado con la que ha dejado de funcionar. El fabricante me envió 1 sin coste alguno para reponer la defectuosa.


vemos la estrella con la que califico al producto comprado

In [12]:
print (f'El comprador {index_random}')
print (f'Califico al producto con {dataset.iloc[index_random].stars} estrellas' )

print(index_random, dataset.iloc[index_random].stars)

El comprador 83031
Califico al producto con 3 estrellas
83031 3


In [13]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.lang.es.examples import sentences

In [14]:
nlp = spacy.load("es_core_news_sm")

In [15]:
doc = nlp (descripcion)

In [16]:
for token in doc:
    print(token.text)

Se
me
ha
fundido
una
el
mismo
día
que
las
he
puesto
.
Iluminar
,
iluminan
bastante
bien
pero
estoy
algo
decepcionado
con
la
que
ha
dejado
de
funcionar
.
El
fabricante
me
envió
1
sin
coste
alguno
para
reponer
la
defectuosa
.


vemos las stop words que tiene identificada Spacy

In [17]:
from spacy.lang.es.stop_words import STOP_WORDS
stopwords_spacy = list(STOP_WORDS)
print (stopwords_spacy)
len(stopwords_spacy)

['mia', 'vosotros', 'antes', 'encuentra', 'lo', 'era', 'se', 'dan', 'considera', 'demasiado', 'último', 'nuestras', 'ultimo', 'consideró', 'aproximadamente', 'dice', 'saben', 'solamente', 'si', 'cuál', 'esa', 'podrian', 'he', 'nuevos', 'serán', 'realizado', 'míos', 'cuánta', 'será', 'su', 'vuestro', 'sí', 'los', 'tiempo', 'trabajas', 'mios', 'indicó', 'otras', 'usa', 'soyos', 'ésos', 'habla', 'menos', 'ella', 'eso', 'salvo', 'tenemos', 'diferente', 'nueva', 'poco', 'propio', 'las', 'han', 'ayer', 'intentas', 'decir', 'unos', 'pocas', 'anterior', 'el', 'quedó', 'dias', 'pasada', 'tiene', 'bueno', 'fueron', 'dia', 'cerca', 'apenas', 'nosotras', 'ver', 'durante', 'ésta', 'buenos', 'todavia', 'fin', 'hizo', 'conseguir', 'entonces', 'eramos', 'esto', 'tengo', 'ninguno', 'existe', 'estos', 'quiza', 'allí', 'cualquier', 'alguno', 'con', 'ésas', 'fuimos', 'aunque', 'podrán', 'tras', 'según', 'segun', 'enseguida', 'ahi', 'cuando', 'sobre', 'consigo', 'sabemos', 'demás', 'él', 'había', 'estuvo',

551

Imprimimos las palabras del texto que no son stop words según Spacy

In [18]:
for token in doc:
    if token.is_stop == False:
        print(token)

fundido
puesto
.
Iluminar
,
iluminan
decepcionado
dejado
funcionar
.
fabricante
envió
1
coste
reponer
defectuosa
.


**Lemmatización**

In [19]:
for token in doc:
    print (token.text, token.lemma_)

Se él
me yo
ha haber
fundido fundir
una uno
el el
mismo mismo
día día
que que
las él
he haber
puesto poner
. .
Iluminar iluminar
, ,
iluminan iluminar
bastante bastante
bien bien
pero pero
estoy estar
algo algo
decepcionado decepcionado
con con
la el
que que
ha haber
dejado dejar
de de
funcionar funcionar
. .
El el
fabricante fabricante
me yo
envió enviar
1 1
sin sin
coste coste
alguno alguno
para para
reponer reponer
la el
defectuosa defectuosa
. .


**POS Part of Speech**

In [20]:
for token in doc:
    print (token.text, token.pos_)

Se PRON
me PRON
ha AUX
fundido VERB
una DET
el DET
mismo DET
día NOUN
que PRON
las PRON
he AUX
puesto VERB
. PUNCT
Iluminar VERB
, PUNCT
iluminan VERB
bastante ADV
bien ADV
pero CCONJ
estoy AUX
algo PRON
decepcionado ADJ
con ADP
la DET
que PRON
ha AUX
dejado VERB
de ADP
funcionar VERB
. PUNCT
El DET
fabricante NOUN
me PRON
envió VERB
1 NUM
sin ADP
coste NOUN
alguno PRON
para ADP
reponer VERB
la DET
defectuosa NOUN
. PUNCT


**Detección de identidades**

In [21]:
displacy.render(doc, style = 'ent')

**Tokenización**

Eliminamos los signos 

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [39]:
import string
puntua = string.punctuation + '¿!¡? + " "'
puntua

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿!¡? + " "'

In [47]:
def text_data_cleaning(sentence):
    doc=nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp= token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
        
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
            
    return clean_tokens
    

In [48]:
text_data_cleaning(descripcion)

['fundir',
 'iluminar',
 'iluminar',
 'decepcionado',
 'dejar',
 'funcionar',
 'fabricante',
 'enviar',
 '1',
 'coste',
 'reponer',
 'defectuosa']

**Vectorización TF-IDF**

In [49]:
from sklearn.svm import LinearSVC

In [50]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [51]:
X = dataset['review_body']
y = dataset['stars']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
X_train.head()

153248            Muy buen tamaño, cumple las expectativas.
67802     A los pocos meses no se podía llamar. Era para...
148889    LO HE USADO POCO PERO DESPRENDE BUEN VOLUMEN D...
103093    Funciona bien, pero devuelto porque la radio e...
104681    Para niños pequeños es un problema el puzzle p...
Name: review_body, dtype: object

In [54]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

##PENDIENTES##

terminar de hacer vectorizacion. 

Pense en dividir el DS en dos. uno que tenga los comentarios de las personas que hayan puntuado entre 1 y 3 estrellas y el otro con el de 4 y 5 y buscar cuales son las palabras que mas se repiten en ambos. Quizas al dividir en dos el df se distribuyen de diferente manera las product_Category y podemos ver si algunas tienen mayor tendencia a tener malas puntuaciones. o cuales son las catergorias con mejor puntuacion.


